# TOC
* [Data collection](#dataCollection)
    * [Retrieve data from Carbon Monitor](#retrieveData)
* [Data preprocessin](#dataPreprocessing)
    * [Prepare the emission dataframe](#emissionDataframe)
    * [Labelling](#labelling)
    * [Convert emissions into matrix](#convertEmission)
    * [Reorder emission matrix](#reorderEmission)
    * [Construct edge_index](#constructEdgeIndex)
    * [Construct edge_attr](#constructEdgeAttr)
* [Modelling construction](#modelling)
    * [Create a dataloader](#createDataloader)
    * [Build the model](#buildModel)
* [Create Train and Evaluation functions](#trainEval)
* [Start training](#training)
* [Test results](#test)
* [Bibliography](#ref)

This assignment collects daily carbon emissions of Chinese provinces from [Carbon Monitor](https://cn.carbonmonitor.org). After preprocessing and labelling, the data is put into **ST-GAT** (Zhang et al., 2019) a Graph Neural Networks (GNNs) model for prediction. The modelling part [Section 4](#modelling) and after was based on [Predicting Los Angeles Traffic with Graphical Neural Networks](https://colab.research.google.com/drive/1NUIQDgj9NXDqtPN_9k_YxKhJN43p3Gho?usp=sharing) which is a Colab version of Zhang et al's work.

Since GNNs requires many packages to be installed beforehand, in case if there is any mismatch that makes some packages fail to be installed in your system, I have restarted Jupyter Kernel and ran all cells with all outputs remained in this notebook. I will also upload another HTMl version which you can refer via [this page](https://palaceparis.github.io/DSAA5020_Assignment1/index.html). The full address is https://palaceparis.github.io/DSAA5020_Assignment1/index.html

# Data collection <a class="anchor" id="dataCollection"></a>

## Retrieve data from [Carbon Monitor](https://cn.carbonmonitor.org) <a class="anchor" id="retrieveData"></a>

In [1]:
import requests
import pandas as pd

# Set the URL of the data to download
url = 'https://datas.carbonmonitor.org/API/downloadFullDataset.php?source=carbon_china'

# Make a GET request to download the data
response = requests.get(url)

# Decode the content of the response as a string
content = response.content.decode('utf-8')

# Convert the string to a StringIO object
from io import StringIO
data = StringIO(content)

# Read the data from the StringIO object into a DataFrame
raw = pd.read_csv(data, delimiter='\t')

In [2]:
raw

,"state,date,sector,value,timestamp"
0,"Anhui,""01/01/2019"",""Residential"",""0.0118734"",""..."
1,"Beijing,""01/01/2019"",""Residential"",""0.300512"",..."
2,"Chongqing,""01/01/2019"",""Residential"",""0"",""1546..."
3,"Fujian,""01/01/2019"",""Residential"",""0"",""1546300..."
4,"Gansu,""01/01/2019"",""Residential"",""0.122791"",""1..."
...,...
226450,"Heilongjiang,""26/09/2022"",""Power"",""0.142754"",""..."
226451,"Heilongjiang,""27/09/2022"",""Power"",""0.144143"",""..."
226452,"Heilongjiang,""28/09/2022"",""Power"",""0.143599"",""..."
226453,"Heilongjiang,""29/09/2022"",""Power"",""0.142346"",""..."


Raw data is collected through simple methods as shown above. They are piled together with ',' as the delimiter. But still, it is easy to see that the data includes provinces and municipalities, date, sectors, corresponding carbon emissions and timestamp. It seems to be recorded from 2019 to 2022.

I am going to use the following variables:

- state
- date
- sector
- value

I expect to label the data by summing the value of all sectors for a province/municipality in each day and predict daily carbon emissions.

# Data prepocessing <a class="anchor" id="dataPreprocessing"></a>

## Prepare the emission dataframe <a class="anchor" id="emissionDataframe"></a>

In [3]:
# split the 'column' column based on "," delimiter
df = raw['state,date,sector,value,timestamp'].str.split(',', expand=True)

# rename the columns
df.columns = ['state', 'date', 'sector', 'value', 'timestamp']

print(df.dtypes)
df

state        object
date         object
sector       object
value        object
timestamp    object
dtype: object


,state,date,sector,value,timestamp
0,Anhui,"""01/01/2019""","""Residential""","""0.0118734""","""1546300800"""
1,Beijing,"""01/01/2019""","""Residential""","""0.300512""","""1546300800"""
2,Chongqing,"""01/01/2019""","""Residential""","""0""","""1546300800"""
3,Fujian,"""01/01/2019""","""Residential""","""0""","""1546300800"""
4,Gansu,"""01/01/2019""","""Residential""","""0.122791""","""1546300800"""
...,...,...,...,...,...
226450,Heilongjiang,"""26/09/2022""","""Power""","""0.142754""","""1664150400"""
226451,Heilongjiang,"""27/09/2022""","""Power""","""0.144143""","""1664236800"""
226452,Heilongjiang,"""28/09/2022""","""Power""","""0.143599""","""1664323200"""
226453,Heilongjiang,"""29/09/2022""","""Power""","""0.142346""","""1664409600"""


In [4]:
df['date'] = df['date'].str.replace('"', '') # remove double quotes
df['sector'] = df['sector'].str.replace('"', '') # remove double quotes
df['value'] = df['value'].str.replace('"', '') # remove double quotes


df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y') # year format
df['value'] = pd.to_numeric(df['value']) # number format

print (df.dtypes)
df

state                object
date         datetime64[ns]
sector               object
value               float64
timestamp            object
dtype: object


,state,date,sector,value,timestamp
0,Anhui,2019-01-01,Residential,0.011873,"""1546300800"""
1,Beijing,2019-01-01,Residential,0.300512,"""1546300800"""
2,Chongqing,2019-01-01,Residential,0.000000,"""1546300800"""
3,Fujian,2019-01-01,Residential,0.000000,"""1546300800"""
4,Gansu,2019-01-01,Residential,0.122791,"""1546300800"""
...,...,...,...,...,...
226450,Heilongjiang,2022-09-26,Power,0.142754,"""1664150400"""
226451,Heilongjiang,2022-09-27,Power,0.144143,"""1664236800"""
226452,Heilongjiang,2022-09-28,Power,0.143599,"""1664323200"""
226453,Heilongjiang,2022-09-29,Power,0.142346,"""1664409600"""


I separate the raw data into different columns and there is still " in some columns so that I am going to remove them and convert *date* and *value* to time value and numeric value respectively.

## Labelling <a class="anchor" id="labelling"></a>

In [5]:
df = df.groupby(['state', 'date'])['value'].sum().reset_index(name='mt_co2_per_day') # emissions from the sum of all sectors
print (df.dtypes)
df

state                     object
date              datetime64[ns]
mt_co2_per_day           float64
dtype: object


,state,date,mt_co2_per_day
0,Anhui,2019-01-01,1.126812
1,Anhui,2019-01-02,1.110479
2,Anhui,2019-01-03,1.078495
3,Anhui,2019-01-04,1.112436
4,Anhui,2019-01-05,1.109971
...,...,...,...
45286,Zhejiang,2022-12-27,1.131461
45287,Zhejiang,2022-12-28,1.140390
45288,Zhejiang,2022-12-29,1.155932
45289,Zhejiang,2022-12-30,1.148535


Since I am going to predict daily carbon emissions for each province/municipality, I sum the value of different sectors and rename it as **mt_co2_per_day**.

The model we are going to use is **Spatio-Temporal GAT** (Zhang et al., 2019). Basically, I view different province/municipality as nodes in GNNs and whether a pair is adjacent will decide if there is an edge to connect the pair. The physical distance between a pair will be considered in the modelling process. This makes sense because if a province presents a high amount of carbon emissions, adajacent regions will also have a high carbon emission, for example, due to their similar industrial structure. After constructing graphs, the prediction will start.

Therefore, the following data is significant for modelling:

- **emission_matrix** which has daily emissions of each province and municipality.
- **edge_index** which records whether there is an edge between a pair.
- **edge_attr** which is the weight of each edge.
- **distance** which records the physical distance between a pair.

To make this project less complicated, and limited by my knowledge, edge_attr will be set as 1 <a class="anchor" id="edge_attr1"></a>.

## Convert emissions into matrix <a class="anchor" id="convertEmission"></a>

In [6]:
data_mat = df.pivot(index='state', columns='date', values='mt_co2_per_day').reset_index()
data_mat.columns.name = None
data_mat = data_mat.rename_axis(None, axis=1)
data_mat

,state,2019-01-01 00:00:00,2019-01-02 00:00:00,2019-01-03 00:00:00,2019-01-04 00:00:00,2019-01-05 00:00:00,2019-01-06 00:00:00,2019-01-07 00:00:00,2019-01-08 00:00:00,2019-01-09 00:00:00,...,2022-12-22 00:00:00,2022-12-23 00:00:00,2022-12-24 00:00:00,2022-12-25 00:00:00,2022-12-26 00:00:00,2022-12-27 00:00:00,2022-12-28 00:00:00,2022-12-29 00:00:00,2022-12-30 00:00:00,2022-12-31 00:00:00
0,Anhui,1.126812,1.110479,1.078495,1.112436,1.109971,1.115140,1.106616,1.121771,1.125911,...,1.167246,1.160539,1.152805,1.170154,1.184461,1.186461,1.196684,1.212896,1.206869,1.176674
1,Beijing,0.486609,0.473364,0.447949,0.449885,0.454155,0.452470,0.447971,0.458651,0.458425,...,0.444162,0.432418,0.427054,0.432654,0.437076,0.441110,0.448208,0.452960,0.441959,0.424388
2,Chongqing,0.292801,0.289154,0.281560,0.290185,0.288968,0.289638,0.288403,0.291965,0.292966,...,0.353799,0.349190,0.346564,0.353195,0.357809,0.358208,0.361068,0.365990,0.363802,0.355032
3,Fujian,0.608134,0.599910,0.583409,0.601722,0.599832,0.602095,0.598343,0.606147,0.608346,...,0.608838,0.602700,0.598392,0.608901,0.616668,0.617460,0.622511,0.630989,0.627546,0.612257
4,Gansu,0.561579,0.550552,0.529533,0.541682,0.542993,0.544607,0.539238,0.548699,0.550043,...,0.533384,0.528641,0.524420,0.531142,0.537145,0.539352,0.545311,0.552209,0.546581,0.530384
5,Guangdong,1.245064,1.230384,1.199000,1.235196,1.229193,1.230903,1.227193,1.241825,1.245938,...,1.431037,1.407239,1.396019,1.425424,1.444590,1.445898,1.457106,1.476995,1.467223,1.432303
6,Guangxi,0.547811,0.540540,0.525827,0.542246,0.540402,0.542253,0.539130,0.546075,0.548031,...,0.793711,0.788647,0.783369,0.795599,0.805440,0.806648,0.813441,0.824508,0.820544,0.800289
7,Guizhou,0.721791,0.711375,0.691051,0.713159,0.711554,0.715100,0.709468,0.719142,0.721861,...,0.615967,0.611787,0.607661,0.617264,0.624919,0.625851,0.631114,0.639697,0.636559,0.620853
8,Hainan,0.072910,0.072052,0.070217,0.072335,0.071982,0.072078,0.071865,0.072721,0.072961,...,0.075734,0.074239,0.073618,0.075292,0.076329,0.076384,0.076961,0.078012,0.077453,0.075631
9,Hebei,3.124042,3.068384,2.962231,3.041952,3.044364,3.057297,3.027868,3.076705,3.086121,...,2.827842,2.811027,2.790546,2.825926,2.858293,2.867818,2.897319,2.934893,2.911451,2.829608


## Reorder emission matrix <a class="anchor" id="reorderEmission"></a>

In [7]:
url= "https://github.com/palaceparis/DSAA5020_Assignment1/blob/main/New_order_of_provinces.csv?raw=true"
new_order = pd.read_csv(url, header = None)
new_order

,0
0,Beijing
1,Tianjin
2,Hebei
3,Shaanxi
4,Inner Mongolia
5,Liaoning
6,Jilin
7,Heilongjiang
8,Shanghai
9,Jiangsu


In [8]:
data_mat.set_index('state', inplace=True) # Set states as index
data_mat

,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,2019-01-09,2019-01-10,...,2022-12-22,2022-12-23,2022-12-24,2022-12-25,2022-12-26,2022-12-27,2022-12-28,2022-12-29,2022-12-30,2022-12-31
state,,,,,,,,,,,,,,,,,,,,,
Anhui,1.126812,1.110479,1.078495,1.112436,1.109971,1.115140,1.106616,1.121771,1.125911,1.145677,...,1.167246,1.160539,1.152805,1.170154,1.184461,1.186461,1.196684,1.212896,1.206869,1.176674
Beijing,0.486609,0.473364,0.447949,0.449885,0.454155,0.452470,0.447971,0.458651,0.458425,0.450730,...,0.444162,0.432418,0.427054,0.432654,0.437076,0.441110,0.448208,0.452960,0.441959,0.424388
Chongqing,0.292801,0.289154,0.281560,0.290185,0.288968,0.289638,0.288403,0.291965,0.292966,0.297832,...,0.353799,0.349190,0.346564,0.353195,0.357809,0.358208,0.361068,0.365990,0.363802,0.355032
Fujian,0.608134,0.599910,0.583409,0.601722,0.599832,0.602095,0.598343,0.606147,0.608346,0.618988,...,0.608838,0.602700,0.598392,0.608901,0.616668,0.617460,0.622511,0.630989,0.627546,0.612257
Gansu,0.561579,0.550552,0.529533,0.541682,0.542993,0.544607,0.539238,0.548699,0.550043,0.553834,...,0.533384,0.528641,0.524420,0.531142,0.537145,0.539352,0.545311,0.552209,0.546581,0.530384
Guangdong,1.245064,1.230384,1.199000,1.235196,1.229193,1.230903,1.227193,1.241825,1.245938,1.265952,...,1.431037,1.407239,1.396019,1.425424,1.444590,1.445898,1.457106,1.476995,1.467223,1.432303
Guangxi,0.547811,0.540540,0.525827,0.542246,0.540402,0.542253,0.539130,0.546075,0.548031,0.557505,...,0.793711,0.788647,0.783369,0.795599,0.805440,0.806648,0.813441,0.824508,0.820544,0.800289
Guizhou,0.721791,0.711375,0.691051,0.713159,0.711554,0.715100,0.709468,0.719142,0.721861,0.734996,...,0.615967,0.611787,0.607661,0.617264,0.624919,0.625851,0.631114,0.639697,0.636559,0.620853
Hainan,0.072910,0.072052,0.070217,0.072335,0.071982,0.072078,0.071865,0.072721,0.072961,0.074131,...,0.075734,0.074239,0.073618,0.075292,0.076329,0.076384,0.076961,0.078012,0.077453,0.075631


In [9]:
data_mat = data_mat.reindex(new_order[0])
data_mat

,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,2019-01-09,2019-01-10,...,2022-12-22,2022-12-23,2022-12-24,2022-12-25,2022-12-26,2022-12-27,2022-12-28,2022-12-29,2022-12-30,2022-12-31
0,,,,,,,,,,,,,,,,,,,,,
Beijing,0.486609,0.473364,0.447949,0.449885,0.454155,0.452470,0.447971,0.458651,0.458425,0.450730,...,0.444162,0.432418,0.427054,0.432654,0.437076,0.441110,0.448208,0.452960,0.441959,0.424388
Tianjin,0.641550,0.626477,0.598088,0.607815,0.611488,0.612466,0.605402,0.617852,0.618761,0.617809,...,0.620317,0.613897,0.608382,0.614586,0.620950,0.624870,0.633161,0.640672,0.631359,0.610009
Hebei,3.124042,3.068384,2.962231,3.041952,3.044364,3.057297,3.027868,3.076705,3.086121,3.122697,...,2.827842,2.811027,2.790546,2.825926,2.858293,2.867818,2.897319,2.934893,2.911451,2.829608
Shaanxi,0.952457,0.934577,0.900272,0.921868,0.923351,0.926094,0.917535,0.933046,0.935460,0.943142,...,1.052822,1.044696,1.036680,1.050047,1.062019,1.065959,1.077317,1.091117,1.081128,1.049927
Inner Mongolia,1.706180,1.673504,1.611893,1.653032,1.656415,1.663735,1.645905,1.674060,1.678893,1.695841,...,1.945186,1.940960,1.927670,1.948057,1.969502,1.976668,1.997668,2.023484,2.008339,1.950909
Liaoning,1.695636,1.657540,1.585613,1.614329,1.622505,1.625764,1.607694,1.639444,1.642302,1.643611,...,1.618128,1.601132,1.586883,1.603878,1.620735,1.630478,1.651617,1.671379,1.647902,1.593102
Jilin,0.643281,0.627110,0.596491,0.603458,0.608021,0.607873,0.601047,0.614247,0.614695,0.610175,...,0.601154,0.591169,0.585084,0.591730,0.597834,0.602263,0.610894,0.617851,0.606572,0.584707
Heilongjiang,0.721820,0.703105,0.667692,0.674391,0.680004,0.679513,0.671748,0.686938,0.687266,0.680764,...,0.707474,0.695695,0.688457,0.695985,0.703066,0.708487,0.718852,0.726963,0.713298,0.687184
Shanghai,0.374075,0.369006,0.358847,0.370115,0.368964,0.370370,0.368043,0.372849,0.374202,0.380756,...,0.322755,0.318909,0.316557,0.322425,0.326598,0.326983,0.329619,0.334111,0.332179,0.324140


In this section, I introduce a new order, reset the index of emission matrix and remove "Tibet" from the dataset. The new order is actually from Figure 1 (Cao et al., 2017) which shows the adjacency information of 31 provinces and cities in China. This order is also used to build edge_index and edge_attr. Again, to make this project less complicated, Tibet is removed with a total of 30 provinces/municipalities left.

<figure>
<img src="https://raw.githubusercontent.com/palaceparis/DSAA5020_Assignment1/main/Geographical-adjacency-information-of-31-provinces-and-cities-in-China_W640.jpg" style="width:80%">
<figcaption align = "center"> Figure 1. Geographical adjacency information of 31 provinces and cities in China. </figcaption>
</figure>


In [10]:
import numpy as np
# np.transpose(data_mat).to_csv('dataset/emissions.csv', index=True, header=True) # write to CSV and upload it onto GitHub for further use.

## Construct edge_index <a class="anchor" id="constructEdgeIndex"></a>

In [11]:
url= "https://github.com/palaceparis/DSAA5020_Assignment1/blob/main/adjacency_information%20.csv?raw=true"
raw_edge = pd.read_csv(url)
print(raw_edge.dtypes)
raw_edge

province     object
x1            int64
x2          float64
x3          float64
x4          float64
x5          float64
x6          float64
x7          float64
x8          float64
dtype: object


,province,x1,x2,x3,x4,x5,x6,x7,x8
0,Beijing,2,3.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Tianjin,1,3.0,15.0,NaN,NaN,NaN,NaN,NaN
2,Hebei,1,2.0,4.0,5.0,6.0,15.0,16.0,NaN
3,Shaanxi,3,5.0,16.0,27.0,NaN,NaN,NaN,NaN
4,Inner Mongolia,3,4.0,6.0,7.0,8.0,27.0,28.0,30.0
5,Liaoning,3,5.0,7.0,NaN,NaN,NaN,NaN,NaN
6,Jilin,5,6.0,8.0,NaN,NaN,NaN,NaN,NaN
7,Heilongjiang,5,7.0,NaN,NaN,NaN,NaN,NaN,NaN
8,Shanghai,10,11.0,NaN,NaN,NaN,NaN,NaN,NaN
9,Jiangsu,9,11.0,12.0,15.0,NaN,NaN,NaN,NaN


The adajacency information is read into Python as shown above. We can know from it that, for example, Beijing is adjacent to Tianjin and Hebei. Same for others.

In [12]:
# create a dictionary to map province names to indices
index_to_province = {i: province for i, province in enumerate(raw_edge['province'])}
index_to_province = {i+1: province for i, province in enumerate(raw_edge['province'])} # Add one to dic
# replace province names with their indices in the raw_edge dataframe
for i in range(31):
    if pd.notnull(raw_edge.loc[i, 'x1']):
        raw_edge.loc[i, 'x1'] = index_to_province[raw_edge.loc[i, 'x1']]
    if pd.notnull(raw_edge.loc[i, 'x2']):
        raw_edge.loc[i, 'x2'] = index_to_province[raw_edge.loc[i, 'x2']]
    if pd.notnull(raw_edge.loc[i, 'x3']):
        raw_edge.loc[i, 'x3'] = index_to_province[raw_edge.loc[i, 'x3']]
    if pd.notnull(raw_edge.loc[i, 'x4']):
        raw_edge.loc[i, 'x4'] = index_to_province[raw_edge.loc[i, 'x4']]
    if pd.notnull(raw_edge.loc[i, 'x5']):
        raw_edge.loc[i, 'x5'] = index_to_province[raw_edge.loc[i, 'x5']]
    if pd.notnull(raw_edge.loc[i, 'x6']):
        raw_edge.loc[i, 'x6'] = index_to_province[raw_edge.loc[i, 'x6']]
    if pd.notnull(raw_edge.loc[i, 'x7']):
        raw_edge.loc[i, 'x7'] = index_to_province[raw_edge.loc[i, 'x7']]
    if pd.notnull(raw_edge.loc[i, 'x8']):
        raw_edge.loc[i, 'x8'] = index_to_province[raw_edge.loc[i, 'x8']]

edge_index = pd.DataFrame(0, index=raw_edge["province"], columns=raw_edge["province"])
for i in range(31):
    for j in range(1, 9):
        if pd.isna(raw_edge.iloc[i, j]):
            pass
        else:
            edge_index.loc[raw_edge.iloc[i, 0], raw_edge.iloc[i, j]] = 1


edge_index

province,Beijing,Tianjin,Hebei,Shaanxi,Inner Mongolia,Liaoning,Jilin,Heilongjiang,Shanghai,Jiangsu,...,Sichuan,Chongqing,Guizhou,Yunnan,Tibet,Shanxi,Gansu,Qinghai,Ningxia,Xinjiang
province,,,,,,,,,,,,,,,,,,,,,
Beijing,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Tianjin,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Hebei,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Shaanxi,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
Inner Mongolia,0,0,1,1,0,1,1,1,0,0,...,0,0,0,0,0,1,1,0,1,0
Liaoning,0,0,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Jilin,0,0,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Heilongjiang,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Shanghai,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


The above shows the edge_index matrix. 1 means adjacent and 0 otherwise. 

In [13]:
from igraph import Graph
edge_index = edge_index.values
g = Graph.Adjacency(edge_index.tolist())
edge_index = g.get_edgelist()
edge_index = np.transpose(edge_index)
edge_index

array([[ 0,  0,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,
         4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  6,  6,  6,  7,  7,
         8,  8,  9,  9,  9,  9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11,
        11, 12, 12, 12, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 15,
        15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17,
        17, 18, 18, 18, 18, 18, 19, 19, 19, 19, 20, 21, 21, 21, 21, 21,
        22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 24,
        25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27,
        27, 27, 28, 28, 28, 28, 29, 29, 29, 30, 30, 30],
       [ 1,  2,  0,  2, 14,  0,  1,  3,  4,  5, 14, 15,  2,  4, 15, 26,
         2,  3,  5,  6,  7, 26, 27, 29,  2,  4,  6,  4,  5,  7,  4,  6,
         9, 10,  8, 10, 11, 14,  8,  9, 11, 12, 13,  9, 10, 13, 14, 15,
        16, 10, 13, 18, 10, 11, 12, 16, 17, 18,  1,  2,  9, 11, 15,  2,
         3, 11, 14, 16, 26, 11, 13, 15, 17, 21, 26, 13, 16, 18, 19, 21,
       

Finally, we convert the matrix into array so that GNNs could read later.

In [14]:
# pd.DataFrame(edge_index).to_csv('edge_index.csv', index=False, header=False) # write to CSV and upload it onto GitHub for further use.

## Construct edge_attr <a class="anchor" id="constructEdgeAttr"></a>

In [15]:
import numpy as np
edge_attr = pd.DataFrame(np.ones((len(np.transpose(edge_index)),1)),columns=['Value'])
edge_attr

,Value
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
135,1.0
136,1.0
137,1.0
138,1.0


As explained in [Section 2.1](#edge_attr1), the value of edge_attr will be 1.

In [16]:
# edge_attr.to_csv('edge_attr.csv', index=False, header=False) # write to CSV and upload it onto GitHub for further use.

The following parts from [Section 4](#modelling) to [Section 6](#training) are based on Zhang et al's work in 2019. I made adjustments accordingly. Basically, [Section 4.1](#creatDataloader) creats a dataloader based on emission_matrix, edge_matrix, edge_attr and distance I consutructed before. Then the model is proposed based on their paper. Finally, a prediction test is conducted.

Currently, due to my limited knowledge, there is not much to explain. You can skip to [Section 7](#test) where prediction performance is plotted for the 30 provinces/municipalities.